In [1]:
from mpgm.mpgm.evaluation.generating_samples import *
from mpgm.mpgm.models.SPGM import SPGM
import matplotlib.pyplot as plt
import numpy as np
import time

import sympy
import cvxopt
import qpsolvers
import qpoases
from quadprog import solve_qp

from typing import Dict, Optional, Any

from mpgm.mpgm.model_fitters.prox_grad_fitters import Prox_Grad_Fitter, Constrained_Prox_Grad_Fitter
from mpgm.mpgm.model_fitters.prox_operators import QuadProgOperator

import scipy
from functools import partial

In [2]:
gg = RandomNmGraphGenerator(10)
wa = Bimodal_Gaussian_Weight_Assigner(mean_1=-0.1,
                                     std_1=0.02,
                                     mean_2=0.1,
                                     std_2=0.02,
                                     split=0.4)
np.random.seed(0)
    
G = gg.generate_graph(nr_variables=10)
wa.assign_weights(G)
model = TPGM(R=10, theta=G)
sampler = TPGMGibbsSampler(burn_in=50, thinning_nr=150)
    
samples = sampler.generate_samples(model, np.zeros((10, )), 150)

/home/mcio/miniconda3/envs/mpgm/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 22401/22401 [00:19<00:00, 1128.91it/s]


In [3]:
for ii in range(10):
    print(G[ii, ii])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [4]:
model = SPGM(R=10, R0=5)

nr_variables = 10
theta_init = np.random.normal(0, 0.1, (nr_variables, nr_variables))
np.fill_diagonal(theta_init, 0)

In [5]:
fitter = Prox_Grad_Fitter(alpha=0.8,
                          accelerated=True,
                          save_regularization_paths=False,
                          init_step_size=0.1,
                          early_stop_criterion='likelihood',
                          keep_diags_zero=True)
# a,b,c,d,e = fitter.fit_node(1, model.calculate_nll, model.calculate_grad_nll, samples, theta_init)
ret = fitter.call_fit_node(model.calculate_nll, model.calculate_grad_nll, samples, theta_init, True)

TypeError: __init__() got an unexpected keyword argument 'keep_diags_zero'

In [ ]:
ret[0]

In [ ]:
from mpgm.mpgm.evaluation.evaluation_metrics import EvalMetrics

In [ ]:
EvalMetrics.calculate_MSEs(ret[0], G, EvalMetrics.SymmModes.NONE)

In [ ]:
nr_variables = 10
nr_iterations = len(e)
print(nr_iterations)
for ii in range(nr_variables):
    plt.plot(list(range(nr_iterations)), e[:,ii])

In [ ]:
# def solve_constrained_prox_grad_gaines(objective:np.ndarray, data:np.ndarray, reg_param:float):
# Decided not to pursue this, as this is a path regularizer + start parameters are initialised
# using QP anyway.

# The question remains: is QP faster than the path method? (even for small params).

In [ ]:
    def prox_operator_cvxopt(data_points:np.ndarray, objective:np.ndarray, reg_parameter:float) -> np.ndarray:
        print('Before: alpha=' + str(reg_parameter) + ' ' + str(objective))
        if True:
            n = len(objective)

            v = objective.reshape((n, 1))
            alpha = reg_parameter
            q = alpha * np.ones((2 * n, 1)) - np.vstack([v, -v])

            # The order of operations is: C -> -C -> (C, -C) -> add one row to it, ensuring positivity of beta aka a
            # row of -1s; d is 2n x 1 of 0s.
            C = data_points
            xC, yC = C.shape
            C = -C  # Since we want our ineq to be greater than or equal to zero.
            G = np.hstack([C, -C])

            # Not sure if this will work.
            G = np.vstack([G, -np.ones((1, 2 * yC))])
            print(G.shape)
            G = Constrained_Prox_Grad_Fitter.remove_dependent_rows(G)
            print(G.shape)
            h = np.zeros((G.shape[0], 1))

            P = np.zeros((2 * n, 2 * n))
            P[0:n, 0:n] = np.identity(n)
            P[n:2 * n, 0:n] = -np.identity(n)
            P[0:n, n:2 * n] = -np.identity(n)
            P[n:2 * n, n:2 * n] = np.identity(n)

            P = cvxopt.matrix(P, tc='d')
            q = cvxopt.matrix(q, tc='d')
            G = cvxopt.matrix(G, tc='d')
            h = cvxopt.matrix(h, tc='d')
            solution = cvxopt.solvers.qp(P, q, G, h, kktsolver='ldl', options={'kktreg': 1e-9, 'show_progress': False,
                                                                               'maxiters': 500})

            beta_2n = np.array(solution['x']).reshape((2 * n,))
            beta = beta_2n[:n] - beta_2n[n:]

            print("After: " + str(beta))
            print("Status: " + str(solution['status'] + ' ' + str(solution['iterations'])))
            return beta, beta_2n

In [ ]:
objective = np.zeros((10, ))
objective[0] = 1
beta = prox_operator_osqp(samples, objective, reg_parameter=1)
print(beta2n)
print(beta)

In [ ]:
def prox_operator_quadprog(data_points:np.ndarray, objective:np.ndarray, reg_parameter:float) -> np.ndarray:
    print('Before: alpha=' + str(reg_parameter) + ' ' + str(objective))
    if True:
        n = len(objective)

        v = objective.reshape((n, 1))
        alpha = reg_parameter
        q = alpha * np.ones((2*n, 1)) - np.vstack([v, -v])
        q = q.reshape((2*n, ))

        # The order of operations is: C -> -C -> (C, -C) -> add one row to it, ensuring positivity of beta aka a
        # row of -1s; d is 2n x 1 of 0s.
        C = data_points
        xC, yC = C.shape
        G = np.hstack([C, -C])

        # Not sure if this will work.
        G = np.vstack([G, np.ones((1, 2 * yC))])
        G = Constrained_Prox_Grad_Fitter.remove_dependent_rows(G)
        h = np.zeros((G.shape[0],))

        P = np.zeros((2*n, 2*n))
        P[0:n, 0:n] = np.identity(n)
        P[n:2*n, 0:n] = -np.identity(n)
        P[0:n, n:2*n] = -np.identity(n)
        P[n:2*n, n:2*n] = np.identity(n)

        print(G.shape)
        solution = solve_qp(P, -q, G.T, h)
        print(solution)

        beta_2n = np.array(solution['x']).reshape((2 * n,))
        beta = beta_2n[:n] - beta_2n[n:]

        print("After: " + str(beta))
        print("Status: " + str(solution['status'] + ' ' + str(solution['iterations'])))
        return beta 


In [ ]:
def prox_operator_qpoases(data_points:np.ndarray, objective:np.ndarray, reg_parameter:float) -> np.ndarray:
    print('Before: alpha=' + str(reg_parameter) + ' ' + str(objective))
    if True:
        n = len(objective)

        v = objective.reshape((n, 1))
        alpha = reg_parameter

        q = alpha * np.ones((2*n, 1)) - np.vstack([v, -v])
        q = q.reshape((2*n, ))

        # The order of operations is: C -> -C -> (C, -C) -> add one row to it, ensuring positivity of beta aka a
        # row of -1s; d is 2n x 1 of 0s.
        C = data_points
        xC, yC = C.shape
        C = -C  # Since we want our ineq to be greater than or equal to zero.
        G = np.hstack([C, -C])

        # Not sure if this will work.
        G = np.vstack([G, -np.identity(2*yC)])
        print(G.shape)
#         G = QuadProgOperator.remove_dependent_rows(G)
#         print(G.shape)
        h = np.zeros((G.shape[0], ))

        P = np.zeros((2 * n, 2 * n))
        P[0:n, 0:n] = np.identity(n)
        P[n:2 * n, 0:n] = -np.identity(n)
        P[0:n, n:2 * n] = -np.identity(n)
        P[n:2 * n, n:2 * n] = np.identity(n)
        
        solution = qpsolvers.qpoases_solve_qp(P, q, G, h)
        print(solution)

        beta_2n = solution.reshape((2 * n,))
        beta = beta_2n[:n] - beta_2n[n:]

        print("After: " + str(beta))
        return beta, beta_2n

In [ ]:
def prox_operator_osqp(data_points:np.ndarray, objective:np.ndarray, reg_parameter:float) -> np.ndarray:
    print('Before: alpha=' + str(reg_parameter) + ' ' + str(objective))
    if True:
        n = len(objective)

        v = objective.reshape((n, 1))
        alpha = reg_parameter

        q = alpha * np.ones((2*n, 1)) - np.vstack([v, -v])
        q = q.reshape((2*n, ))

        # The order of operations is: C -> -C -> (C, -C) -> add one row to it, ensuring positivity of beta aka a
        # row of -1s; d is 2n x 1 of 0s.
        C = data_points
        xC, yC = C.shape
        C = -C  # Since we want our ineq to be greater than or equal to zero.
        G = np.hstack([C, -C])

        # Not sure if this will work.
        G = np.vstack([G, -np.identity(2*yC)])
        print(G.shape)
#         G = Constrained_Prox_Grad_Fitter.remove_dependent_rows(G)
#         print(G.shape)
        h = np.zeros((G.shape[0], ))

        P = np.zeros((2*n, 2*n))
        P[0:n, 0:n] = np.identity(n)
        P[n:2 * n, 0:n] = -np.identity(n)
        P[0:n, n:2 * n] = -np.identity(n)
        P[n:2 * n, n:2 * n] = np.identity(n)
        
        solution = qpsolvers.osqp_solve_qp(P, q, G, h)
        print(solution)

        beta_2n = solution.reshape((2 * n,))
        beta = beta_2n[:n] - beta_2n[n:]

        print("After: " + str(beta))
        return beta, beta_2n

In [ ]:
np.identity(3)

In [ ]:
objective = np.zeros((10, ))
objective[0] = 1
beta, beta2n = prox_operator_qpoases(samples, objective, reg_parameter=2)
print(beta2n)
print(beta)

In [ ]:
# Need to write + test a projection operator onto the intersection of halfspaces.
# scipy.optimize.minimize? How fast is it?
def l2norm(x:np.ndarray, objective:np.ndarray, reg_param:float) -> float:
    sq_norm = (x-objective)**2
    return np.sum(sq_norm) * 1/(2 * reg_param)

def jac_l2norm(x:np.ndarray, objective:np.ndarray, reg_param:float) -> np.ndarray:
    return (x-objective)/reg_param

# def constraint(x:np.ndarray, data_point:np.ndarray, node):
#     inequality_lh = np.dot(x, data_point) - x[node] * data_point[node] + x[node]
#     return inequality_lh >= 0

# def constraint_jac(x:np.ndarray, data_point:np.ndarray, node:int) -> np.ndarray:
#     jac = np.array(data_point)
#     jac[node] = 1
#     return jac

# def construct_constraints(data:np.ndarray, node:int) -> Dict[Any, Any]:
#     dict_seq = []
#     reduced_data = Constrained_Prox_Grad_Fitter.remove_dependent_rows(data)
#     for ii in range(reduced_data.shape[0]):
#         data_point = reduced_data[ii, :]
#         constraint_dict = dict()
#         constraint_dict['type'] = 'ineq'
#         constraint_dict['fun'] = partial(constraint, data_point=data_point, node=node)
#         constraint_dict['jac'] = partial(constraint_jac, data_point=data_point, node=node)
#         dict_seq.append(constraint_dict)
#     return dict_seq

def construct_constraints(data:np.ndarray, node:int):
    A = np.copy(data)
    A[:, node] = 1
    constraints = scipy.optimize.LinearConstraint(A, 0, np.inf, keep_feasible=False)
    return constraints

def soft_thresholding_operator(objective:np.ndarray, reg_parameter:float) -> np.ndarray:
    obj_plus = objective - reg_parameter
    obj_minus = objective + reg_parameter
    return obj_plus * (obj_plus > 0) + obj_minus * (obj_minus < 0)

# Disagree with this.
def check_convergence(x:np.ndarray, z:np.ndarray, threshold:Optional[float]=1e-3):
    converged_params = (x - z) ** 2 <= (threshold ** 2) * (x ** 2)
    all_converged = all(converged_params)
    return all_converged

In [ ]:
def prox_operator_admm(data_points:np.ndarray, 
                       objective:np.ndarray, 
                       reg_parameter:float, 
                       node:int, 
                       tau:Optional[float]=0.1,
                       min_iter:Optional[int]=100,
                       max_iter:Optional[int]=1000) -> np.ndarray:
    n = len(objective)
    u = np.zeros((n,))
    z = np.copy(objective)
    x = np.copy(objective)
    
    new_reg_parameter = reg_parameter/(1+tau)
    
    f_operator = partial(soft_thresholding_operator, 
                         reg_parameter=new_reg_parameter)
    
    # Need to specify: x0, jac and fun.
    g_operator = partial(scipy.optimize.minimize, 
                         method="SLSQP",
                         constraints=construct_constraints(data_points, node))
    
    # Found bug: this flavor of ADMM makes x converge to -z. 
    # Since z is the result of g_operator, keeping constraints true for x, means
    # multiplying them by -1 to keep them true for z (if it makes sense).
    
    # STOOOPID.
    
    iteration = 0
    while (iteration <= min_iter or not check_convergence(x, z)) and iteration <= max_iter:
        print(iteration)
        
        prior_lasso_objective = (objective + tau*(z-u))/(1+tau)
        
        print(prior_lasso_objective)
        
        x = f_operator(prior_lasso_objective)
        
        fun = partial(l2norm, objective=x+u, reg_param=1.0)
        jac = partial(jac_l2norm, objective=x+u, reg_param=1.0)
        z = g_operator(x0=(x+u), fun=fun, jac=jac)['x']
        
        u = u + x - z
        
        iteration += 1
        
        print(x)
        print(z)
    
    return x, z

In [ ]:
objective = np.random.normal(0, 0.1, (10, ))
objective

In [ ]:
x, z = prox_operator_admm(data_points=samples,
                      objective=objective,
                      reg_parameter=0.01,
                      node=0,
                      max_iter=100)

In [ ]:
x

In [ ]:
z

In [ ]:
x + z

In [ ]:
from scipy.optimize import linprog
import numpy as np

c = np.zeros(6)
c[5] = 1
A = np.array([[1, 2, 7, 2, 4],
             [0, 0, 9, 6, 2],
             [7, 9, 4, 5, 3],
             [1, 4, 0, 7, 9],
             [9, 7, 3, 8, 3]])
A_ub = np.hstack((A, -np.ones((5,1))))
b_ub = np.zeros(5)

A_eq = np.ones((1, 6))
A_eq[0, 5] = 0
b_eq = np.ones(1)

# Default bounds are (0, None).
x = linprog(c, A_ub, b_ub, A_eq, b_eq)
print(x)

In [ ]:
import numpy as np

In [ ]:
def prox(objective: np.ndarray, reg_parameter: float, node: int, data_points: np.ndarray) -> np.ndarray:
    obj_plus = objective - reg_parameter
    obj_minus = objective + reg_parameter
    return obj_plus * (obj_plus > 0) + obj_minus * (obj_minus < 0)

In [ ]:
s = np.array([0.5, 0.6, 0.7, 1])
print(prox(s, 0.6, 0, np.array([])))

In [ ]:
L = np.identity(10)

In [ ]:
L[9,0] = 1
L[8,3] = 1

In [ ]:
L

In [ ]:
L * L.T

In [ ]:
from sklearn.datasets import make_sparse_spd_matrix

In [ ]:
A = make_sparse_spd_matrix(dim=10,
                          alpha=0.9,
                          norm_diag=False,
                          smallest_coef=0.1,
                          largest_coef=0.1)

In [ ]:
print(A)